In [1]:
from hawk.keygen import hawkkeygen_unpacked as hku
from ntrugen import karamul, gen_poly
import numpy as np
from falcon import Falcon
from fft import add, mul

In [2]:
class PolyMatrix2x2:
    """
    2x2 matrix of polynomials in Z[x]/(x^n + 1)
    Determinant must be 1.
    Singleton: only one instance allowed.
    """

    def __init__(self, a, b, c, d):
        """
        Matrix:
            [ a  b ]
            [ c  d ]
        Each entry is a polynomial (list of ints).
        """
        self.a = a
        self.b = b
        self.c = c
        self.d = d

    # ---------------------------------------------------------
    # Determinant
    # ---------------------------------------------------------
    def det(self):
        """
        det = a*d - b*c  (mod x^n + 1)
        """
        ad = karamul(self.a, self.d)
        bc = karamul(self.b, self.c)
        return [ad[i] - bc[i] for i in range(len(ad))]

    def _det_is_one(self):
        det = self.det()
        expected = [1] + [0] * (len(det) - 1)
        return det == expected

    # ---------------------------------------------------------
    # Inverse (since det = 1)
    # ---------------------------------------------------------
    def inverse(self):
        """
        Since det = 1:

            M^{-1} =
            [  d  -b ]
            [ -c   a ]
        """
        minus_b = [-x for x in self.b]
        minus_c = [-x for x in self.c]

        return PolyMatrix2x2(self.d, minus_b, minus_c, self.a)

    # ---------------------------------------------------------
    # Matrix multiplication (uses karamul)
    # ---------------------------------------------------------
    def __matmul__(self, other):
        """
        Matrix multiplication using karamul.
        """
        if not isinstance(other, PolyMatrix2x2):
            raise TypeError("Can only multiply with another PolyMatrix2x2")

        a = [karamul(self.a, other.a)[i] + karamul(self.b, other.c)[i]
             for i in range(len(self.a))]

        b = [karamul(self.a, other.b)[i] + karamul(self.b, other.d)[i]
             for i in range(len(self.a))]

        c = [karamul(self.c, other.a)[i] + karamul(self.d, other.c)[i]
             for i in range(len(self.a))]

        d = [karamul(self.c, other.b)[i] + karamul(self.d, other.d)[i]
             for i in range(len(self.a))]

        return PolyMatrix2x2(a, b, c, d)

    # ---------------------------------------------------------
    # Pretty polynomial printer
    # ---------------------------------------------------------
    @staticmethod
    def _poly_to_str(p):
        terms = []
        for i, coef in enumerate(p):
            if coef == 0:
                continue
            if i == 0:
                terms.append(str(coef))
            elif i == 1:
                terms.append(f"{coef}x")
            else:
                terms.append(f"{coef}x^{i}")
        return " + ".join(terms) if terms else "0"

    # ---------------------------------------------------------
    # Representation
    # ---------------------------------------------------------
    def __repr__(self):
        return (
            "[ "
            + self._poly_to_str(self.a) + "    "
            + self._poly_to_str(self.b) + " ]\n"
            "[ "
            + self._poly_to_str(self.c) + "    "
            + self._poly_to_str(self.d) + " ]"
        )

In [4]:
def rand_poly_2x2_gen():
    n1 = [1] + 255 * [0]
    n2, n3 = gen_poly(256), gen_poly(256)
    temp = karamul(n2, n3)
    temp[0] += 1
    n4 = temp
    return PolyMatrix2x2(n1, n2, n3, n4)

In [5]:
f, g, F, G = hku(8)[:4]

In [6]:
B = PolyMatrix2x2(f, g, F, G)

In [7]:
U1 = rand_poly_2x2_gen()
xw = B.__matmul__(U1)

In [8]:
assert xw._det_is_one() == True

In [9]:
U2 = rand_poly_2x2_gen()
zw = B.__matmul__(U2)

In [12]:
assert zw._det_is_one() == True

In [13]:
zw_inv = zw.inverse()

### Rounding params
$$p_1 = (20109 · (2 · 256)) + 1$$
$$q = 2^{32}$$
$$p = 2^5$$

In [75]:
p1 = 20109 * 2 * 256 + 1
q = 1 << 32
p = 1 << 5

In [15]:
falcon = Falcon(512)

In [20]:
sk, vk = falcon.keygen()

In [21]:
ig = falcon.sign(sk, b'testing falcon sign')

In [22]:
falcon.verify(vk, b'testing falcon sign', ig)

True

### Generating xtoken

In [23]:
xtoken = xw.__matmul__(zw)

In [32]:
def lwr(lst, q, p):
    """
    Apply LWR rounding from modulus q to modulus p.
    
    For each a in lst:
        a <- a mod q
        return round(p * a / q) mod p
    """
    result = []
    for a in lst:
        a_mod = a % q
        # print(a_mod)
        scaled = (a_mod * p) / q
        rounded = int(round(scaled)) % p
        # print(scaled, rounded)
        result.append(rounded)
        # return
    return result

In [62]:
a, b, c, d = lwr(xtoken.a, q, p1), lwr(xtoken.b, q, p1), lwr(xtoken.c, q, p1), lwr(xtoken.d, q, p1)
xtoken_r = PolyMatrix2x2(a, b, c, d)

In [26]:
rsign = falcon.raw_sign(sk, b'testing falcon sign') ## this gives a vector of 2 polynomials

In [27]:
temp_poly = PolyMatrix2x2(rsign[0], 256 * [0], rsign[1], 256 * [0])

In [28]:
xtag = xw.__matmul__(temp_poly)

### Generating xtag

In [76]:
a, b, c, d = lwr(xtag.a, q, p), lwr(xtag.b, q, p), lwr(xtag.c, q, p), lwr(xtag.d, q, p)
xtag_r = PolyMatrix2x2(a, b, c, d)

In [29]:
ytag = zw_inv.__matmul__(temp_poly)

### Generating ytag

In [64]:
a, b, c, d = lwr(ytag.a, q, p1), lwr(ytag.b, q, p1), lwr(ytag.c, q, p1), lwr(ytag.d, q, p1)
ytag_r = PolyMatrix2x2(a, b, c, d)

### Generating xtoken * ytag

In [65]:
hypoth = xtoken_r.__matmul__(ytag_r)

In [77]:
a, b, c, d = lwr(hypoth.a, p1, p), lwr(hypoth.b, p1, p), lwr(hypoth.c, p1, p), lwr(hypoth.d, p1, p)
hypoth_r = PolyMatrix2x2(a, b, c, d)

In [78]:
xtag_r

[ 1x^104 + 31x^122 + 31x^173 + 31x^222    0 ]
[ 1 + 1x^2 + 1x^3 + 1x^4 + 1x^7 + 31x^9 + 1x^10 + 31x^11 + 30x^12 + 31x^14 + 31x^15 + 30x^16 + 31x^18 + 30x^20 + 31x^21 + 31x^23 + 31x^25 + 31x^27 + 31x^28 + 1x^29 + 31x^30 + 1x^31 + 31x^32 + 1x^36 + 1x^37 + 1x^38 + 1x^39 + 1x^40 + 1x^41 + 1x^43 + 1x^44 + 2x^45 + 31x^46 + 1x^47 + 2x^50 + 2x^51 + 1x^52 + 31x^53 + 1x^55 + 31x^56 + 1x^59 + 31x^60 + 31x^62 + 31x^65 + 31x^67 + 30x^68 + 31x^70 + 1x^71 + 30x^72 + 31x^74 + 31x^75 + 30x^76 + 1x^78 + 31x^81 + 1x^83 + 31x^84 + 1x^86 + 2x^87 + 1x^90 + 1x^91 + 1x^92 + 31x^93 + 1x^95 + 1x^99 + 30x^100 + 1x^101 + 1x^102 + 31x^104 + 1x^106 + 31x^107 + 1x^109 + 1x^110 + 30x^112 + 1x^113 + 31x^116 + 1x^118 + 30x^119 + 1x^122 + 1x^125 + 31x^126 + 2x^129 + 1x^130 + 1x^133 + 1x^134 + 31x^135 + 1x^136 + 1x^137 + 1x^138 + 1x^141 + 31x^142 + 31x^144 + 31x^145 + 31x^149 + 1x^150 + 31x^151 + 31x^152 + 30x^154 + 31x^155 + 31x^158 + 30x^159 + 1x^160 + 31x^163 + 31x^164 + 31x^167 + 31x^168 + 31x^170 + 1x^171 + 31x^172 

In [79]:
hypoth_r

[ 17 + 22x + 9x^2 + 9x^3 + 4x^4 + 4x^5 + 23x^6 + 13x^7 + 21x^8 + 10x^9 + 8x^10 + 14x^11 + 7x^12 + 16x^13 + 15x^14 + 8x^15 + 9x^16 + 3x^17 + 15x^18 + 14x^19 + 14x^20 + 13x^21 + 6x^22 + 4x^23 + 1x^24 + 17x^25 + 18x^26 + 15x^27 + 7x^28 + 2x^29 + 26x^30 + 21x^31 + 18x^32 + 24x^33 + 9x^34 + 7x^35 + 15x^36 + 13x^37 + 30x^38 + 9x^39 + 16x^41 + 13x^43 + 12x^44 + 25x^45 + 7x^46 + 1x^47 + 25x^48 + 7x^49 + 13x^50 + 26x^51 + 18x^52 + 5x^53 + 7x^54 + 5x^55 + 27x^56 + 31x^57 + 24x^58 + 29x^59 + 21x^60 + 26x^61 + 2x^62 + 12x^63 + 28x^64 + 19x^65 + 5x^66 + 13x^67 + 13x^68 + 7x^69 + 19x^70 + 14x^71 + 3x^72 + 18x^73 + 28x^74 + 27x^75 + 12x^76 + 19x^77 + 12x^78 + 4x^79 + 1x^80 + 23x^81 + 13x^82 + 2x^83 + 5x^84 + 1x^85 + 24x^86 + 30x^87 + 14x^88 + 28x^89 + 17x^90 + 31x^91 + 4x^92 + 30x^93 + 9x^94 + 13x^95 + 3x^96 + 18x^97 + 28x^98 + 14x^99 + 13x^100 + 30x^101 + 29x^102 + 28x^103 + 12x^104 + 28x^105 + 17x^106 + 6x^107 + 8x^109 + 8x^110 + 14x^111 + 8x^112 + 29x^113 + 25x^114 + 23x^115 + 8x^116 + 22x^117 + 4